<a href="https://colab.research.google.com/github/harsha-bsm/Topic-Modelling-using-LDA-and-NMF-reviewsflipkart/blob/master/POS_TAGGING_only_NOUNS_LDA_7830.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import numpy as np
import spacy
import pandas as pd

In [2]:
data=pd.read_csv("mi7830.csv")
dataoriginal=pd.read_csv("mi7830.csv")

In [3]:
pip install emot 

In [4]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [5]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub("\S*\d\S*", "", phrase).strip()
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [6]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [7]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too',"ok", 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile","np","pro","k20"])

In [8]:
from nltk.corpus import stopwords
import nltk

In [9]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [10]:
def lengthgreaterthantwo(review):
  return " ".join([word for word in review.split(" ") if  len(word) > 2])

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation,NMF

In [12]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

## POS TAGGING taking only nouns

In [13]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')
def only_nouns(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if pos=="NN"])
  return lis
  
data["review"]=data["review"].apply(lambda x:only_nouns(x))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
data["review"]

0       display type phone camera performance time ama...
1       camera performance competition priceVery displ...
2       review month budget phone phone hand. phone bu...
3       phone battery camera front camera night mode p...
4       review use performance charm .not second lag.C...
                              ...                        
7825                           superb mobile, value money
7826                     Camera price segment. weight bit
7827                                            SWITCHING
7828                                                     
7829                                                issue
Name: review, Length: 7830, dtype: object

In [18]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:lengthgreaterthantwo(x)))

In [19]:
data["review"]=pd.DataFrame(data["review"].apply(lambda x:lengthgreaterthantwo(x)))

In [20]:
data["review"][2]

'review month budget hand budget difference everything quality display glass body design gorilla glass front divine lags management heating stock android bit segment classy history book time camera sensor everything camera setup bit front camera camera sensor budget image processing note mode average time day use charge charger min information cons fingerprint scanner protector camera bit face irritated notification light time'

### LDA BOW ---  (topipc extraction/keyword extraction)

In [21]:
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=3,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),8)

[['price',
  'camera',
  'flagship',
  'range',
  'performance',
  'pubg',
  'superb',
  'display'],
 ['camera',
  'performance',
  'battery',
  'price',
  'display',
  'quality',
  'backup',
  'delivery'],
 ['money',
  'camera',
  'value',
  'value money',
  'quality',
  'display',
  'performance',
  'battery']]

In [22]:
 topics=print_top_words(lda,bow.get_feature_names(),6)
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,dominant_topic,review,dominant_topic_keywords
0,0.029435,0.927565,0.043000,1,It's awesome display not so new type but phone...,"[camera, performance, battery, price, display,..."
1,0.022190,0.147771,0.830039,2,Excellent camera excellent performance ..Redmi...,"[money, camera, value, value money, quality, d..."
2,0.002904,0.003126,0.993971,2,Giving my review after 1 month of usage. Hope ...,"[money, camera, value, value money, quality, d..."
3,0.029380,0.939151,0.031469,1,nice phone battery is good camera is good fron...,"[camera, performance, battery, price, display,..."
4,0.044045,0.002529,0.953426,2,"Detailed review after 5 days of use K20 Pro , ...","[money, camera, value, value money, quality, d..."
...,...,...,...,...,...,...
7825,0.358553,0.056550,0.584897,2,"superb mobile, value for money","[money, camera, value, value money, quality, d..."
7826,0.035329,0.929578,0.035093,1,Camera could have been little better with this...,"[camera, performance, battery, price, display,..."
7827,0.664827,0.167557,0.167616,0,VERY BAD USER INTERFACE. SAMSUNG GALAXY M10 IS...,"[price, camera, flagship, range, performance, ..."
7828,0.333333,0.333333,0.333333,0,Not As Per Acception...,"[price, camera, flagship, range, performance, ..."


In [23]:
modelcomponents=lda.components_/(np.sum(lda.components_,axis=1)).reshape(-1,1)
pd.DataFrame(modelcomponents,index=topic,columns=bow.get_feature_names())

,aap,aap time,ability,ability say,absence,absence blaster,absence marketing,absence memory,absence port,absence slot,absence stabilizing,absence videos,absolute,absolute beast,absolute end,absolute joy,absolute stunner,absolutely,abt,abt abt,abt case,abt unlock,accent,accent battery,accepting,accepting popup,access,access apps,access brother,access case,access wide,accha,accha maza,account,account light,account logout,accuracy,accuracy apps,accuracy battery,accuracy rate,...,yhatsbthe,yhatsbthe part,youtube,youtube automatically,youtube cant,youtube dont,youtube support,yrrr,yrrr result,yrs,yrs camera,yrs releasing,yrs thought,yrs usage,yrs yrs,zabardasstt,zabardasstt camera,zing,zing zing,zone,zone display,zone superb,zoom,zoom angle,zoom camera,zoom clarity,zoom image,zoom modetree,zoom option,zoom popup,zoom portraits,zoom quality,zoom sense,zoom shot,zoom suggest,zoom super,zoom zoom,zooming,zooming delay,zooming range
topic_0,0.000068,0.000068,0.000068,0.000017,0.000251,0.000017,0.000068,0.000017,0.000068,0.000018,0.000068,0.000062,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,...,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000120,0.000068,0.000069,0.000017,0.000068,0.000080,0.000017,0.000068,0.000066,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017
topic_1,0.000012,0.000012,0.000012,0.000012,0.000097,0.000047,0.000012,0.000047,0.000012,0.000012,0.000012,0.000016,0.000046,0.000048,0.000012,0.000012,0.000012,0.000012,0.000083,0.000047,0.000012,0.000047,0.000047,0.000012,0.000012,0.000012,0.000047,0.000012,0.000012,0.000047,0.000012,0.000012,0.000012,0.000058,0.000012,0.000045,0.000083,0.000012,0.000047,0.000012,...,0.000012,0.000012,0.000048,0.000012,0.000047,0.000012,0.000012,0.000047,0.000047,0.000047,0.000012,0.000012,0.000047,0.000012,0.000012,0.000047,0.000047,0.000012,0.000012,0.000047,0.000047,0.000012,0.000251,0.000012,0.000012,0.000013,0.000047,0.000047,0.000012,0.000047,0.000012,0.000047,0.000012,0.000047,0.000012,0.000047,0.000012,0.000083,0.000047,0.000047
topic_2,0.000013,0.000013,0.000050,0.000050,0.000015,0.000013,0.000013,0.000013,0.000013,0.000050,0.000013,0.000013,0.000165,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000013,0.000050,0.000013,0.000051,0.000050,0.000050,0.000050,0.000163,0.000050,0.000050,0.000013,0.000050,0.000050,0.000050,0.000076,0.000050,0.000015,0.000088,0.000050,0.000013,0.000050,...,0.000050,0.000050,0.000163,0.000050,0.000013,0.000050,0.000050,0.000013,0.000013,0.000164,0.000050,0.000050,0.000013,0.000050,0.000050,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000315,0.000050,0.000050,0.000013,0.000013,0.000013,0.000050,0.000013,0.000050,0.000050,0.000050,0.000013,0.000050,0.000013,0.000050,0.000013,0.000013,0.000013


### LDA TFIDF

In [24]:
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(tfidf_words)
topics=print_top_words(lda,words,15)

In [25]:
pd.DataFrame(topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,price,superb,awesome,camera,battery,quality,camera quality,worth,smartphone,performance,backup,look,good,pubg,battery backup
1,money,value,value money,excellent,quality,device,issue,delivery,redmi,camera,love,price,wow,display,problem
2,super,camera,osm,performance,everything,battery,budget,time,quality,word,service,display,mind,price,buy
3,camera,performance,flagship,nice,range,price,beast,killer,experience,price range,display,camera performance,segment,flagship killer,battery


In [26]:
topic=["topic_"+str(i) for i  in range(0,lda.n_components)]
df_doc_topic=pd.DataFrame(lda.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["proreview"]=data["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]


In [27]:
df_doc_topic["review_proc"]=data["review"]
df_doc_topic["dominant_topic"].value_counts()

0    3417
3    1853
1    1402
2    1158
Name: dominant_topic, dtype: int64

In [28]:
subtopic0=df_doc_topic.loc[df_doc_topic["dominant_topic"]==0]["review_proc"]
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(subtopic0)
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=2,random_state=0)
lda.fit_transform(tfidf_words)
topics=print_top_words(lda,words,15)
pd.DataFrame(topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,price,awesome,worth,look,camera,good,money,worth money,flipkart,flagship,killer,performance,deal,best,value
1,superb,camera,battery,quality,camera quality,performance,smartphone,backup,display,pubg,battery backup,everything,penny,market,till


In [29]:
subtopic1=df_doc_topic.loc[df_doc_topic["dominant_topic"]==1]["review_proc"]
tfidf=TfidfVectorizer(ngram_range=(1,3))
tfidf_words=tfidf.fit_transform(subtopic1)
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=3,random_state=0)
lda.fit_transform(tfidf_words)
topics=print_top_words(lda,words,9)
pd.DataFrame(topics)

,0,1,2,3,4,5,6,7,8
0,issue,perfect,camera,cost,performance,problem,beauty,quality,display
1,delivery,redmi,wow,purchase,camera,price,buying,day,battery
2,money,value,value money,excellent,quality,device,love,camera,price


### NMF BOW 


In [30]:
bow=CountVectorizer(ngram_range=(1,3))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=3, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
pd.DataFrame(print_top_words(nmf,bow_words,8))

,0,1,2,3,4,5,6,7
0,camera,performance,quality,battery,display,time,price,screen
1,price,performance,flagship,battery,range,display,device,killer
2,camera,money,value,value money,nice,issue,everything,front


In [31]:
topics=print_top_words(nmf,bow_words,5)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(np.round(nmf.transform(bow_review),2),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,dominant_topic,review,dominant_topic_keywords
0,0.09,0.00,0.00,0,It's awesome display not so new type but phone...,"[camera, performance, quality, battery, display]"
1,0.12,0.00,0.04,0,Excellent camera excellent performance ..Redmi...,"[camera, performance, quality, battery, display]"
2,1.08,0.00,0.00,0,Giving my review after 1 month of usage. Hope ...,"[camera, performance, quality, battery, display]"
3,0.09,0.00,0.00,0,nice phone battery is good camera is good fron...,"[camera, performance, quality, battery, display]"
4,0.06,1.64,0.00,1,"Detailed review after 5 days of use K20 Pro , ...","[price, performance, flagship, battery, range]"
...,...,...,...,...,...,...
7825,0.00,0.00,0.07,2,"superb mobile, value for money","[camera, money, value, value money, nice]"
7826,0.07,0.01,0.00,0,Camera could have been little better with this...,"[camera, performance, quality, battery, display]"
7827,0.00,0.00,0.00,0,VERY BAD USER INTERFACE. SAMSUNG GALAXY M10 IS...,"[camera, performance, quality, battery, display]"
7828,0.00,0.00,0.00,0,Not As Per Acception...,"[camera, performance, quality, battery, display]"


In [32]:
df_doc_topic["review_proc"]=data["review"]
df_doc_topic["dominant_topic"].value_counts()

0    4139
1    2392
2    1299
Name: dominant_topic, dtype: int64

In [34]:
subtopic0=df_doc_topic.loc[df_doc_topic["dominant_topic"]==0]["review_proc"]
bow=CountVectorizer(ngram_range=(1,2))
bow_review=bow.fit_transform(subtopic0)

nmf = NMF(n_components=2, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
pd.DataFrame(print_top_words(nmf,bow_words,10))

,0,1,2,3,4,5,6,7,8,9
0,camera,performance,quality,display,battery,price,camera quality,camera performance,device,superb
1,battery,issue,time,display,super,screen,fingerprint,mode,life,problem


In [35]:
subtopic1=df_doc_topic.loc[df_doc_topic["dominant_topic"]==1]["review_proc"]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(subtopic1)

nmf = NMF(n_components=2, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,10)

[['performance',
  'battery',
  'flagship',
  'display',
  'device',
  'killer',
  'delivery',
  'pubg',
  'experience',
  'smartphone'],
 ['price',
  'range',
  'beast',
  'segment',
  'exchange',
  'value',
  'buy',
  'deal',
  'wow',
  'date']]

### NMF TFIDF

In [36]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [37]:

nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.01,
          l1_ratio=.5).fit(tfidf_words)
topics=print_top_words(nmf,words,10)
pd.DataFrame(topics)

,0,1,2,3,4,5,6,7,8,9
0,money,value,value money,excellent,experience,delivery,worth,bit,worth money,xiaomi
1,camera,quality,battery,awesome,display,issue,camera quality,everything,backup,problem
2,price,flagship,range,price range,killer,budget,smartphone,segment,flagship killer,price segment
3,performance,beast,camera performance,redmi,performance camera,battery,penny,battery performance,performance battery,display
4,super,superb,nice,device,flipkart,service,delivery,till,fast,cost


In [38]:
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(nmf.fit_transform(tfidf_words),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,dominant_topic,review,dominant_topic_keywords
0,3.937378e-03,0.048384,0.00000,0.179224,0.000000e+00,3,It's awesome display not so new type but phone...,"[performance, beast, camera performance, redmi..."
1,2.339830e-01,0.000018,0.00000,0.027924,0.000000e+00,0,Excellent camera excellent performance ..Redmi...,"[money, value, value money, excellent, experie..."
2,1.709441e-24,0.434723,0.00000,0.000000,0.000000e+00,1,Giving my review after 1 month of usage. Hope ...,"[camera, quality, battery, awesome, display, i..."
3,5.372138e-36,0.135369,0.00000,0.000000,0.000000e+00,1,nice phone battery is good camera is good fron...,"[camera, quality, battery, awesome, display, i..."
4,7.346840e-04,0.097673,0.69079,0.052266,3.967854e-18,2,"Detailed review after 5 days of use K20 Pro , ...","[price, flagship, range, price range, killer, ..."


In [39]:
df_doc_topic["review_proc"]=data["review"]
df_doc_topic["dominant_topic"].value_counts()

0    3144
1    2213
2    1085
3     820
4     568
Name: dominant_topic, dtype: int64

In [40]:
subtopic0=df_doc_topic.loc[df_doc_topic["dominant_topic"]==0]["review_proc"]
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(subtopic0)
nmf = NMF(n_components=2, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
words=tfidf.get_feature_names()
pd.DataFrame(print_top_words(nmf,words,10))

,0,1,2,3,4,5,6,7,8,9
0,money,value,value money,experience,worth,camera,battery,performance,worth money,bit
1,excellent,delivery,time,order,company,today,boy,flipcart,money excellent,time delivery


In [41]:
subtopic1=df_doc_topic.loc[df_doc_topic["dominant_topic"]==1]["review_proc"]
tfidf=TfidfVectorizer(ngram_range=(1,2))
tfidf_words=tfidf.fit_transform(subtopic1)
nmf = NMF(n_components=2, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
words=tfidf.get_feature_names()
pd.DataFrame(print_top_words(nmf,words,10))

,0,1,2,3,4,5,6,7,8,9
0,camera,quality,display,everything,camera quality,problem,time,pubg,screen,look
1,awesome,battery,issue,backup,battery backup,word,life,camera battery,heating,update


In [42]:
data.to_csv("mi7830_preprocess.csv",index=False)